<a href="https://colab.research.google.com/github/Priyanshu-Naik/Gen_AI/blob/main/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation in Python

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam

In [10]:
from google.colab import files
uploaded = files.upload()

Saving daily_temperature_100.csv to daily_temperature_100.csv


In [12]:
import io

df = pd.read_csv(io.StringIO(uploaded['daily_temperature_100.csv'].decode('utf-8')))
print(df.head())

         date  temperature
0  2010-01-01        20.99
1  2010-01-02        20.76
2  2010-01-03        22.35
3  2010-01-04        24.51
4  2010-01-05        23.87


Preprocessing the Data

We will scale our data to ensure all features have equal weight and avoid any bias. In this example, we will use MinMaxScaler, which scales the data to a range between 0 and 1. Proper scaling is important because neural networks tend to perform better when input features are normalized.




In [13]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['temperature'].values.reshape(-1, 1))

Preparing Data for GRU

We will define a function to prepare our data for training our model.

create_dataset(): Prepares the dataset for time-series forecasting. It creates sliding windows of time_step length to predict the next time step.

X.reshape(): Reshapes the input data to fit the expected shape for the GRU which is 3D: [samples, time steps, features].

In [15]:
def create_dataset(data, time_step=1):
  X, y = [], []
  for i in range(len(data)-time_step-1):
    X.append(data[i:(i+time_step), 0])
    y.append(data[i + time_step, 0])
  return np.array(X), np.array(y)

time_step = 10
X, y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

Building the GRU Model

We will define our GRU model with the following components:

GRU(units=50): Adds a GRU layer with 50 units (neurons).

return_sequences=True: Ensures that the GRU layer returns the entire sequence (required for stacking multiple GRU layers).

Dense(units=1): The output layer which predicts a single value for the next time step.

Adam(): An adaptive optimizer commonly used in deep learning.

In [17]:
model = Sequential()
model.add(GRU(50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(GRU(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.2837
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1987
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1326
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1012
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1034
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1094
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1038
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0946
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0838
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0776
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0716
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0710
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0694
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0682
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0655
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━━━

In [18]:
input_sequences = scaled_data[-time_step:].reshape(1, time_step, 1)
predicted_temperature = model.predict(input_sequences)
predicted_temperature = scaler.inverse_transform(predicted_temperature)
print("Predicted Temperature:", predicted_temperature)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
Predicted Temperature: [[20.366423]]
